# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster() #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
# TO-DO: Create a Keyspace 
# FROM Module 2 - Chapter 15 - Exercise 3 Solution
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = { 
            'class'              : 'SimpleStrategy', 
            'replication_factor' : 1 
        } 
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# TO-DO: Set KEYSPACE to the keyspace specified above
# FROM Module 2 - Chapter 15 - Exercise 3 Solution
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [7]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Drop table if it already exists just to make sure we can overwrite it if needed
query = """DROP TABLE IF EXISTS event;"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create Table
query = """
    CREATE TABLE IF NOT EXISTS events
    (session_id int, items_in_session int, artist varchar, song varchar, length float, 
        PRIMARY KEY ((session_id), items_in_session));
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
query = """
            INSERT INTO events
            (session_id, items_in_session, artist, song, length)
            VALUES (%s, %s, %s, %s, %s);
        """

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## Extract the relevant fields from the CSV file line
        artist           = line[0]
        items_in_session = line[3]
        length           = line[5]
        session_id       = line[8]
        song             = line[9]
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(session_id), int(items_in_session), artist, song, float(length) ))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist, song, length FROM events 
        WHERE session_id       = 338 
        AND   items_in_session = 4;
"""
try:
    query_hit = session.execute(query)
except Exception as e:
    print(e)
    
for hit in query_hit:
    print(hit.artist, "-", hit.song, "-", hit.length)

Faithless - Music Matters (Mark Knight Dub) - 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create the Table

# Drop table if it already exists just to make sure we can overwrite it if needed
query = """DROP TABLE IF EXISTS more_events;"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create Table
query = """
    CREATE TABLE IF NOT EXISTS more_events
    (user_id int, session_id int, items_in_session int, 
    first_name varchar, last_name varchar, artist varchar, song varchar, 
        PRIMARY KEY ((user_id, session_id), items_in_session))
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Fill table using the CSV FILE
file = 'event_datafile_new.csv'
query = """
    INSERT INTO more_events
        (user_id, session_id, items_in_session, first_name, last_name, artist, song)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
        """

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## Extract the relevant fields from the CSV file line
        artist           = line[0]
        first_name       = line[1]
        items_in_session = line[3]
        last_name        = line[4]
        session_id       = line[8]
        song             = line[9]
        user_id          = line[10]
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (int(user_id), int(session_id), int(items_in_session), first_name, last_name, artist, song ))
        except Exception as e:
            print(e)


In [18]:
# Extract the desired fields from the table
query = """
    SELECT artist, song, first_name, last_name FROM more_events
        WHERE user_id    = 10 
        AND   session_id = 182
    ORDER BY items_in_session;
"""
try:
    query_hit = session.execute(query)
except Exception as e:
    print(e)
    
for hit in query_hit:
    print(hit.artist, "-", hit.song, "-", hit.first_name, hit.last_name)
                    

Down To The Bone - Keep On Keepin' On - Sylvie Cruz
Three Drives - Greece 2000 - Sylvie Cruz
Sebastien Tellier - Kilometer - Sylvie Cruz
Lonnie Gordon - Catch You Baby (Steve Pitron & Max Sanna Radio Edit) - Sylvie Cruz


In [32]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create the Table

# Drop table if it already exists just to make sure we can overwrite it if needed
query = """DROP TABLE IF EXISTS even_more_events"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Create Table
query = """
    CREATE TABLE IF NOT EXISTS even_more_events
    (song varchar, artist varchar, user_id int, first_name varchar, last_name varchar, 
        PRIMARY KEY (song, artist, user_id));
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Fill table using the CSV FILE
file = 'event_datafile_new.csv'
query = """
    INSERT INTO even_more_events (song, artist, user_id, first_name, last_name) 
        VALUES (%s, %s, %s, %s, %s);
"""

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        ## Extract the relevant fields from the CSV file line
        artist           = line[0]
        first_name       = line[1]
        last_name        = line[4]
        song             = line[9]
        user_id          = line[10]
        
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query, (song, artist, int(user_id), first_name, last_name))
        except Exception as e:
            print(e)


In [34]:
# Select the desired entiers
query = """
    SELECT first_name, last_name FROM even_more_events
        WHERE song = 'All Hands Against His Own';
"""
try:
    query_hit = session.execute(query)
except Exception as e:
    print(e)
    
for hit in query_hit:
    print(hit.first_name, hit.last_name)

Jacqueline - Lynch
Tegan - Levine
Sara - Johnson


### Drop the tables before closing out the sessions

In [37]:
## TO-DO: Drop the table before closing out the sessions
tables = ["events", "more_events", "event_more_events"]
query  = """DROP TABLE IF EXISTS {}"""
for table in tables:
    try:
        session.execute(query.format(table))
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()